In [1]:
import findspark

In [2]:
findspark.init()
findspark.find()

'C:\\install\\spark-3.1.2-bin-hadoop3.2'

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *

spark = (
    SparkSession.builder.appName("firstpyspark").master("local[4]").getOrCreate()
)
spark

In [26]:
from pyspark.sql.types import StructType, StructField, IntegerType

schema = StructType([StructField("Id", IntegerType(), True)])

In [27]:
numbers = [[1],[2],[3]]

# Define schema as a list of tuples or a StructType
#schema = ["Id"]  # Simple list of column names

# Create DataFrame
numbersDF = spark.createDataFrame(numbers, schema)
numbersDF.show()

Traceback (most recent call last):
  File "C:\install\spark-3.1.2-bin-hadoop3.2\python\pyspark\serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\install\spark-3.1.2-bin-hadoop3.2\python\pyspark\cloudpickle\cloudpickle_fast.py", line 72, in dumps
    cp.dump(obj)
  File "C:\install\spark-3.1.2-bin-hadoop3.2\python\pyspark\cloudpickle\cloudpickle_fast.py", line 540, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\install\spark-3.1.2-bin-hadoop3.2\python\pyspark\cloudpickle\cloudpickle_fast.py", line 630, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\install\spark-3.1.2-bin-hadoop3.2\python\pyspark\cloudpickle\cloudpickle_fast.py", line 503, in _function_reduce
    return self._dynamic_function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\install\spark-3.1.2-bin-hadoo

PicklingError: Could not serialize object: IndexError: tuple index out of range

In [11]:
df =  spark.read.csv(r"C:\Users\shravanr\learning\spark\input_dir\a.csv")


df.show()

+---+
|_c0|
+---+
|1	a|
|2	b|
|3	c|
+---+



In [12]:
df = spark.read.format("csv") \
    .option("delimiter", "\t") \
    .option("header", "false") \
    .load(r"C:\Users\shravanr\learning\spark\input_dir\a.csv")

df = df.withColumnRenamed("_c0", "column1") \
       .withColumnRenamed("_c1", "column2") \


df.show()

+-------+-------+
|column1|column2|
+-------+-------+
|      1|      a|
|      2|      b|
|      3|      c|
+-------+-------+



In [13]:
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as F

# Create a SparkSession
spark = SparkSession.builder.appName("ReadMultiValuedTSV").getOrCreate()

# Read the TSV file (ensure correct path)
df = spark.read.format("csv") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .option("header", "true") \
    .load(r"C:\Users\shravanr\learning\spark\input_dir\BrandsPrototype.tsv")



# Assuming 'multi_value_column' is the name of the column with multi-valued data
df = df.withColumn("BrandAliases", F.split(df["BrandAliases"], ","))
df = df.withColumn("SampleProductNames", F.split(df["SampleProductNames"], ","))
df = df.withColumn("TopRetailers", F.split(df["TopRetailers"], ","))


df.show()

+-------+---------+------------+--------------+-----------+------------------+------------+------------+
|BrandId|BrandName|BrandAliases|       Domains|NumProducts|SampleProductNames|NumRetailers|TopRetailers|
+-------+---------+------------+--------------+-----------+------------------+------------+------------+
|      1|        a|      [a, b]|a.com,a.com.au|         40|      [d, f, c, j]|          23|    [14, 67]|
+-------+---------+------------+--------------+-----------+------------------+------------+------------+



In [14]:
df.printSchema()

root
 |-- BrandId: integer (nullable = true)
 |-- BrandName: string (nullable = true)
 |-- BrandAliases: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- Domains: string (nullable = true)
 |-- NumProducts: integer (nullable = true)
 |-- SampleProductNames: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- NumRetailers: integer (nullable = true)
 |-- TopRetailers: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [15]:
import pysolr

In [16]:
solr = pysolr.Solr('http://localhost:8983/solr/Brands', always_commit=False)

In [19]:
batch_size = 1000

def index_batch(batch):
    """Index a batch of documents to Solr."""
    # Convert Spark Rows to dictionaries
    solr_ = pysolr.Solr('http://localhost:8983/solr/Brands', always_commit=False)
    documents = [row.asDict() for row in batch]
    
    if documents:
        solr_.add(documents)
        print(f"Indexed batch of {len(documents)} documents.")

# Process DataFrame in batches and index
df.foreachPartition(index_batch)

# Commit changes to Solr
#solr.commit()
print("Data ingestion completed.")


Traceback (most recent call last):
  File "C:\install\spark-3.1.2-bin-hadoop3.2\python\pyspark\serializers.py", line 437, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\install\spark-3.1.2-bin-hadoop3.2\python\pyspark\cloudpickle\cloudpickle_fast.py", line 72, in dumps
    cp.dump(obj)
  File "C:\install\spark-3.1.2-bin-hadoop3.2\python\pyspark\cloudpickle\cloudpickle_fast.py", line 540, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\install\spark-3.1.2-bin-hadoop3.2\python\pyspark\cloudpickle\cloudpickle_fast.py", line 630, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\install\spark-3.1.2-bin-hadoop3.2\python\pyspark\cloudpickle\cloudpickle_fast.py", line 503, in _function_reduce
    return self._dynamic_function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\install\spark-3.1.2-bin-hadoo

PicklingError: Could not serialize object: IndexError: tuple index out of range